In [1]:
# import the opencv library
import cv2
from viewer import Viewer

In [2]:
viewer = Viewer()

In [3]:
import torch
import os
import numpy as np

from models import Net
import math

In [4]:
def get_model(i):
    checkpoint_dir = '/mnt/heap/My folder/tune_reports/best_var/find_hyperparameters_ac795_00000_0_2022-03-26_03-46-41/checkpoint_0000'
    s = '0' + str(i) if i < 10 else str(i)
    checkpoint_dir += s
    
    torch_model = Net()
    checkpoint = os.path.join(checkpoint_dir, "checkpoint")
    model_state, optimizer_state = torch.load(checkpoint)
    torch_model.load_state_dict(model_state)
    torch_model = torch_model.eval() 
    return torch_model

In [5]:
model = get_model(49)

In [6]:
import torchvision.transforms as T
img_size = (254, 254)
train_transforms = T.Compose([
        T.Resize(img_size),
        T.ToTensor(),
        T.Normalize(mean=[0.43, 0.44, 0.47],
                    std=[0.20, 0.20, 0.20])
    ])

In [7]:
def predict(frame):
    
    height, width, depth = frame.shape
    img = viewer.convert_from_cv2_to_image(frame)
    x_data = train_transforms(img)
    pred = model(x_data.unsqueeze(0))
    if len(pred) < 1:
        return
    if torch.round(pred['class']) == 1:
        box = pred['bbox'][0]
        box = viewer.convert_box(box,width,height)

        frame = cv2.rectangle(frame, (box[0], box[1]), (box[2], box[3]),
                                (0, 0, 255), 4)
    return frame    

In [8]:

  
  
# define a video capture object
vid = cv2.VideoCapture(0)
try:
    while(True):

        # Capture the video frame
        # by frame
        ret, frame = vid.read()
        if type(frame) == 'NoneType':
            continue

        # Display the resulting frame
        frame = predict(frame)
        cv2.imshow('frame', frame)

        # the 'q' button is set as the
        # quitting button you may use any
        # desired button of your choice
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
finally:  
    # After the loop release the cap object
    vid.release()
    # Destroy all the windows
    cv2.destroyAllWindows()